# Wrapping an existing model
SALib performs sensitivity analysis for any model that can be expressed in the form of `f(X)=Y`,
where `X` is a matrix of inputs (often referred to as the model's factors)

The analysis methods are independent of the model and can be applied non-intrusively such that it does not matter what
`f` is.

Typical model implementations take the form of `f(a, b, c, ...)=Y`. In other words, each model factor is supplied as a separate argument to the function. In such cases it is necessary to write a wrapper to allow use with SALib. This is illustrated here with a simple linear function:

In [ ]:
def linear(a, b, x):
    """
    Return y = a + b + x
    """
    return a + b + x

As SALib expects a (numpy) matrix of factors, we simply "wrap" the function above like so:

In [ ]:
def wrapped_linear(X, func = linear):
    """g(X) = Y, where X := [a b x] and g(X) := f(X)"""
    # We transpose to obtain each column (the model factors) as separate variables
    a, b, X = X.T
    
    # Then call the original model
    return func(a, b, x)

# Parallel evaluation and analysis
Here we expand on some technical details that enable parallel evaluation and analysis. We noted earlier that
the model being "wrapped" is also passed in as an argument. This is to facilitate parallel evaluation, as the
arguments to the wrapper are passed on to workers. The approach works be using Python's motable default argument
behavior.

A further consideration is that imported modules/packages are not made available to workers in cases where
functions are defined in the same file SALib is used in. Running the previous example with
`.evaluate(wrapped_linear, nprocs=2)` will fail with `NameError: name 'np' is not defined.`

The quick fix is to re-import the required packages within the model function itself:

In [ ]:
def wrapped_linear(X: np.ndarray, func=linear) -> np.ndarray:
    import numpy as np  # re-import necessary packages

    N, D = X.shape
    results = np.empty(N)
    for i in range(N):
        a, b, x = X[i, :]
        results[i] = func(a, b, x)

    return results

This can, however, get unwieldy for complicated models. The recommended best practice is to separate 
implementation (i.e., model definitions) from its use. Simply moving the model functions into a
separate file is enough for this example, such that the project structure is something like:

project_directory:  
| -- model_definition.py  
|____analyais.py

In [1]:
import SALib
print(SALib.__version__)

1.4.7
